<a href="https://colab.research.google.com/github/JanChocyk/Laptops_data_analyst/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import plotly.graph_objects as go
import plotly.offline as po
from plotly.subplots import make_subplots
from datetime import datetime
import pandas as pd
import argparse
import pickle
import os

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
import models

In [ ]:
MODEL_NAME = 'ModelVGGSmall'
LEARNING_RATE = 0.001
EPOCHS = 15
BATCH_SIZE = 32
INPUT_SHAPE = (224, 224, 3)
CLASES = 58
TRAIN_DIR = '/content/drive/MyDrive/Colab Notebooks/traffic_sign_classifier/data/train'
VALID_DIR = '/content/drive/MyDrive/Colab Notebooks/traffic_sign_classifier/data/valid'

In [ ]:
def plot_hist(history, filename):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    fig = make_subplots(rows=2, cols=1, subplot_titles=('Accuracy', 'Loss'))

    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['accuracy'], name='train_accuracy',
                             mode='markers+lines', marker_color='#f29407'), row=1, col=1)
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_accuracy'], name='valid_accuracy',
                             mode='markers+lines', marker_color='#0771f2'), row=1, col=1)
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['loss'], name='train_loss',
                             mode='markers+lines', marker_color='#f29407'), row=2, col=1)
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_loss'], name='valid_loss',
                             mode='markers+lines', marker_color='#0771f2'), row=2, col=1)

    fig.update_xaxes(title_text='Liczba epok', row=1, col=1)
    fig.update_xaxes(title_text='Liczba epok', row=2, col=1)
    fig.update_yaxes(title_text='Accuracy', row=1, col=1)
    fig.update_yaxes(title_text='Loss', row=2, col=1)
    fig.update_layout(width=1400, height=1000, title=f"Metrics: {MODEL_NAME}")

    po.plot(fig, filename=filename, auto_open=False)

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=30,
    rescale=1. / 255.,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255.)

train_generator = train_datagen.flow_from_directory(
    directory=TRAIN_DIR,
    target_size=INPUT_SHAPE[:2],
    batch_size=BATCH_SIZE
)

valid_generator = valid_datagen.flow_from_directory(
    directory=VALID_DIR,
    target_size=INPUT_SHAPE[:2],
    batch_size=BATCH_SIZE
)

Found 3336 images belonging to 58 classes.
Found 860 images belonging to 58 classes.


In [ ]:
architecture = models.ModelVGGNetSmall(input_shape=INPUT_SHAPE, num_classes=CLASES, final_activation='sigmoid')
model_vgg = architecture.build()

In [ ]:
model_vgg.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model_vgg.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 220, 220, 64)      18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 110, 110, 64)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 110, 110, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 108, 108, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 106, 106, 128)     147584    
                                                        

In [ ]:
if not os.path.exists('/content/drive/MyDrive/Colab Notebooks/traffic_sign_classifier/output_model_vgg'):
    os.mkdir('/content/drive/MyDrive/Colab Notebooks/traffic_sign_classifier/output_model_vgg')
dt = datetime.now().strftime('%d_%m_%Y_%H_%M')
filepath = os.path.join('/content/drive/MyDrive/Colab Notebooks/traffic_sign_classifier/output_model_vgg', 'model_' + dt + '.hdf5')
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath, monitor='val_accuracy', save_best_only=True)

In [ ]:
history_vgg = model_vgg.fit_generator(
    generator=train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[checkpoint]
)

<ipython-input-9-5ebcf218117d>:1: UserWarning:

`Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.



Epoch 1/30
104/104 [==============================] - 424s 4s/step - loss: 3.5517 - accuracy: 0.1979 - val_loss: 2.6858 - val_accuracy: 0.3329
Epoch 2/30
104/104 [==============================] - 102s 981ms/step - loss: 2.5663 - accuracy: 0.3057 - val_loss: 2.3816 - val_accuracy: 0.3618
Epoch 3/30
104/104 [==============================] - 102s 985ms/step - loss: 2.3038 - accuracy: 0.3354 - val_loss: 2.0522 - val_accuracy: 0.4038
Epoch 4/30
104/104 [==============================] - 81s 780ms/step - loss: 2.1597 - accuracy: 0.3771 - val_loss: 1.9607 - val_accuracy: 0.4014
Epoch 5/30
104/104 [==============================] - 90s 866ms/step - loss: 2.0203 - accuracy: 0.3919 - val_loss: 1.8586 - val_accuracy: 0.4339
Epoch 6/30
104/104 [==============================] - 106s 1s/step - loss: 1.9478 - accuracy: 0.4186 - val_loss: 1.7124 - val_accuracy: 0.4748
Epoch 7/30
104/104 [==============================] - 102s 981ms/step - loss: 1.8295 - accuracy: 0.4389 - val_loss: 1.6172 - val_acc

In [ ]:
filename = os.path.join('/content/drive/MyDrive/Colab Notebooks/traffic_sign_classifier/output_model_vgg', 'report_' + dt + '.html')
plot_hist(history_vgg, filename=filename)

In [ ]:
architecture = models.ModelResNet50(input_shape=INPUT_SHAPE, num_classes=CLASES, final_activation='sigmoid')
model_rn = architecture.build()

In [ ]:
model_rn.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model_rn.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [ ]:
if not os.path.exists('/content/drive/MyDrive/Colab Notebooks/traffic_sign_classifier/output_model_rn'):
    os.mkdir('/content/drive/MyDrive/Colab Notebooks/traffic_sign_classifier/output_model_rn')
dt = datetime.now().strftime('%d_%m_%Y_%H_%M')
filepath = os.path.join('/content/drive/MyDrive/Colab Notebooks/traffic_sign_classifier/output_model_rn', 'model_' + dt + '.hdf5')
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath, monitor='val_accuracy', save_best_only=True)

In [ ]:
history_rn = model_rn.fit_generator(
    generator=train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[checkpoint]
)

<ipython-input-9-ba501194d4ad>:1: UserWarning:

`Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.



Epoch 1/15
104/104 [==============================] - 2688s 25s/step - loss: 16.5393 - accuracy: 0.0850 - val_loss: 4.5479 - val_accuracy: 0.0721
Epoch 2/15
104/104 [==============================] - 71s 674ms/step - loss: 3.9044 - accuracy: 0.1625 - val_loss: 4.2764 - val_accuracy: 0.0349
Epoch 3/15
104/104 [==============================] - 72s 693ms/step - loss: 2.5162 - accuracy: 0.3287 - val_loss: 3.4167 - val_accuracy: 0.1647
Epoch 4/15
104/104 [==============================] - 79s 755ms/step - loss: 2.1871 - accuracy: 0.4001 - val_loss: 28.9034 - val_accuracy: 0.1779
Epoch 5/15
104/104 [==============================] - 73s 705ms/step - loss: 1.9602 - accuracy: 0.4331 - val_loss: 1.8795 - val_accuracy: 0.4483
Epoch 6/15
104/104 [==============================] - 79s 758ms/step - loss: 1.6321 - accuracy: 0.5012 - val_loss: 1.4571 - val_accuracy: 0.5349
Epoch 7/15
104/104 [==============================] - 73s 701ms/step - loss: 1.4548 - accuracy: 0.5451 - val_loss: 1.2995 - val_

In [ ]:
filename = os.path.join('/content/drive/MyDrive/Colab Notebooks/traffic_sign_classifier/output_model_rn', 'report_' + dt + '.html')
plot_hist(history_rn, filename=filename)